In [82]:
import pandas as pd
from joblib import load

In [83]:
call = pd.read_csv("../data/Validation_call.txt", sep='\t')

HER2_chr = 17
HER2_start = 35076296

In [84]:
HER2_status = call[(call['Chromosome']==HER2_chr) & (call['Start']==HER2_start)]\
    .drop(columns=['Chromosome','Start','End','Nclone']).squeeze(axis=0) == 2

In [85]:
X = call[HER2_status[~HER2_status].index].T
model = load('../model/model.pkl')
y_pred = model.predict(X).astype(bool)

prediction_HR = pd.Series('HR+', index=X.index[y_pred])
prediction_TN = pd.Series('Triple Neg', index=X.index[~y_pred])
prediction_HER2 = pd.Series('HER2+', index=HER2_status[HER2_status].index)

predictions_all = pd.concat([prediction_HR,prediction_TN,prediction_HER2])[call.columns[4:]]

In [86]:
test_accuracy = 0.81
predicted_true = round(prediction_HER2.shape[0]+test_accuracy*X.shape[0])

with open('../results/estimate.txt', "w") as f:
    print(predicted_true,file=f)

In [87]:
predictions_all.to_csv('../results/prediction.txt',sep='\t',
                       index_label='Sample',header=['Subgroup'],quoting=1)